In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import keras_tuner as kt

2024-06-07 01:12:47.221382: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
raw_train_set, raw_valid_set= tfds.load(
    name="imdb_reviews",
    split=["train[:90%]", "train[90%:]"],
    as_supervised=True
)

tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)

for review, label in raw_train_set.take(4):
    print(review.numpy().decode("utf-8")[:200], "...")
    print("Label:", label.numpy())


2024-06-07 01:12:49.672535: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-07 01:12:49.877438: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-07 01:12:49.880192: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0
Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Moun ...
Label: 0
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful perf ...
Label: 1


2024-06-07 01:12:51.499954: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-06-07 01:12:51.500637: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [3]:
vocab_size = 1000
text_vec_layer = layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))

def model_builder(hp):
    model = keras.Sequential()
    model.add(text_vec_layer)
    model.add(layers.Embedding(vocab_size, hp.Int('embed_size', min_value=64, max_value=256, step=32), mask_zero=True))
    
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.GRU(units=hp.Int(f'unit_{i}', min_value=64, max_value=256, step=32), return_sequences=True))
    
    model.add(layers.GRU(units=hp.Int('final_units', min_value=64, max_value=256, step=32)))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])
    
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])

    if hp_optimizer == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate)
    elif hp_optimizer == 'sgd':
        optimizer = keras.optimizers.SGD(learning_rate=hp_learning_rate)
    elif hp_optimizer == 'rmsprop':
        optimizer = keras.optimizers.RMSprop(learning_rate=hp_learning_rate)
    
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model


2024-06-07 01:12:56.816361: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [5]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=20,  # Increase the max_epochs
    factor=3,
    directory='my_dir',
    project_name='imdb_reviews'
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

tuner.search(train_set, validation_data=valid_set, epochs=20, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first GRU layer is {best_hps.get('unit_0')} with embedding size {best_hps.get('embed_size')} and final units {best_hps.get('final_units')}.
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')} with optimizer {best_hps.get('optimizer')}.
""")


Trial 30 Complete [00h 16m 13s]
val_accuracy: 0.4927999973297119

Best val_accuracy So Far: 0.8776000142097473
Total elapsed time: 02h 28m 25s

The hyperparameter search is complete. The optimal number of units in the first GRU layer is 128 with embedding size 64 and final units 160.
The optimal learning rate for the optimizer is 0.001 with optimizer rmsprop.



In [6]:
# Build and train the final model with the optimal hyperparameters
model = tuner.hypermodel.build(best_hps)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    'IMDBModel.keras',
    save_best_only=True,
    monitor='val_accuracy',
    verbose=1
)

early_stop_cb = tf.keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True,
    monitor='val_loss',
    verbose=1
)

history = model.fit(
    train_set,
    validation_data=valid_set,
    epochs=30,
    callbacks=[checkpoint_cb, early_stop_cb]
)

# Save the final model
model.save('savedModel.keras')


Epoch 1/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.5281 - loss: 0.6808
Epoch 1: val_accuracy improved from -inf to 0.67040, saving model to IMDBModel.keras
704/704 ━━━━━━━━━━━━━━━━━━━━ 68s 92ms/step - accuracy: 0.5282 - loss: 0.6807 - val_accuracy: 0.6704 - val_loss: 0.6522
Epoch 2/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.7534 - loss: 0.5242
Epoch 2: val_accuracy improved from 0.67040 to 0.77760, saving model to IMDBModel.keras
704/704 ━━━━━━━━━━━━━━━━━━━━ 63s 90ms/step - accuracy: 0.7534 - loss: 0.5241 - val_accuracy: 0.7776 - val_loss: 0.4721
Epoch 3/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.8074 - loss: 0.4457
Epoch 3: val_accuracy did not improve from 0.77760
704/704 ━━━━━━━━━━━━━━━━━━━━ 63s 90ms/step - accuracy: 0.8074 - loss: 0.4456 - val_accuracy: 0.6600 - val_loss: 0.5893
Epoch 4/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.8359 - loss: 0.3838
Epoch 4: val_accuracy improved from 0.77760 to 0.85520, saving model to